In [1]:
import os
import re

import nltk

# download the stopwords and wordnet corpus
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

from nltk.corpus import stopwords
stopword = stopwords.words('english')
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


import gensim
from gensim.models import KeyedVectors
from gensim.utils import simple_preprocess   


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ravi\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ravi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ravi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# change working directory
import os
print('Before chdir:', os.getcwd())

os.chdir('F:\\cosinesim\\') 
         
print('After chdir:', os.getcwd())

os.listdir()

Before chdir: C:\Users\Ravi
After chdir: F:\cosinesim


['A TUTORIAL ON DISTANCE METRIC LEARNING.pdf',
 'cosine similarity-Copy2.ipynb',
 'Cosine_Similarity_with_Glove.ipynb',
 'embeddings.xlsx',
 'fasttext_model300.mod',
 'fasttext_model300.mod.vectors.npy',
 'glovemodel.mod',
 'glovemodel.mod.vectors.npy',
 'Semantic-Similarity-of-Sentences-master.zip',
 'text_similarity_solution .ipynb',
 'w2vecmodel.mod',
 'w2vecmodel.mod.vectors.npy']

In [3]:
%%time
# Preprocessing 
def tokenize(text):
    text.lower()
    
    # use library re to replace urls by token - urlplaceholder
#     text = re.sub(url_regex, 'urlplaceholder', text)
    
    # use library re to replace non ascii characters by a space
#     text = re.sub(non_ascii_regex, ' ', text)

    # use word_tokenize to tokenize the sentences
    tokens = word_tokenize(text)
    
    # instantiate an object of class WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()

    # use a list comprehension to lemmatize the tokens and remove the the stopwords
    clean_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stopword]

    # return the tokens
    return clean_tokens

Wall time: 0 ns


In [4]:
%%time
# data - sentences
query = "apple is fruit"
doc = "apple is phone"

Wall time: 0 ns


In [5]:
%%time
import re
query_tokens = tokenize(query)
print("query tokens", query_tokens)

doc_tokens = tokenize(doc)
print("doc tokens", doc_tokens)

query tokens ['apple', 'fruit']
doc tokens ['apple', 'phone']
Wall time: 2.84 s


In [11]:
%%time

# caluclate similarity - fasttext
import gensim
from gensim.models import KeyedVectors
from gensim.utils import simple_preprocess   

def get_sentence_similarity(sentence_1, sentence_2, model_wv):
    tokens_1 = tokenize(sentence_1)    
    tokens_2 = tokenize(sentence_2)    
    return model_wv.n_similarity(tokens_1, tokens_2)

fasttext_model300 = gensim.models.KeyedVectors.load("./fasttext_model300.mod")
sim = get_sentence_similarity('apple is fruit', 'apple is phone', fasttext_model300)
print(sim)

0.7750639
Wall time: 1min 44s
Parser   : 258 ms


In [12]:
%%time

# caluclate similarity - glove
import gensim
from gensim.models import KeyedVectors
  
def get_sentence_similarity(sentence_1, sentence_2, model_wv):
    tokens_1 = tokenize(sentence_1)    
    tokens_2 = tokenize(sentence_2)    
    return model_wv.n_similarity(tokens_1, tokens_2)

glove_model = gensim.models.KeyedVectors.load("./glovemodel.mod")
sim = get_sentence_similarity('apple is fruit', 'apple is phone', glove_model)
print(sim)

0.7795816
Wall time: 51.9 s
Compiler : 538 ms
Parser   : 481 ms


In [15]:
%%time

# caluclate similarity - w2vec
import gensim
from gensim.models import KeyedVectors

def get_sentence_similarity(sentence_1, sentence_2, model_wv):
    tokens_1 = tokenize(sentence_1)    
    tokens_2 = tokenize(sentence_2)    
    return model_wv.n_similarity(tokens_1, tokens_2)


v2w_model = gensim.models.KeyedVectors.load("./w2vecmodel.mod")
sim = get_sentence_similarity('apple is fruit', 'apple is phone', glove_model)
print(sim)

0.7795816
Wall time: 12min 14s


What is doc2vec?
doc2vec like word2vec creates numerical representation of sentence/paragraphs/documents.
Doc2Vec computes a feature vector for every document in the corpus.
The vectors generated by doc2vec can be used for tasks like finding 
similarity between sentences/paragraphs/documents

In [68]:
%%time

# caluclate similarity - doc2vec
# import gensim doc2vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

def Docs2Vec(s1, s2):
    
    # Create docs
    docs = [s1, s2]
    
    # tokenization of docs
    tokenized_docs = []
    for doc in docs:
        tokenized_docs.append(tokenize(doc))
    print("tokenized_docs:", tokenized_docs)
    
    # tagging docs
    tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate(tokenized_docs)]
#     print("Tagged data:", tagged_data)
    
    
    ## Train doc2vec model
    model = Doc2Vec(tagged_data, vector_size = 20, window = 2, min_count = 1, epochs = 100)

    '''
    vector_size = Dimensionality of the feature vectors.
    window = The maximum distance between the current and predicted word within a sentence.
    min_count = Ignores all words with total frequency lower than this.
    alpha = The initial learning rate.
    '''

    ## Print model vocabulary
#     print("model vocabulary:", model.wv.vocab)
    
    
    cosim = model.wv.n_similarity(query_tokens, doc_tokens)
    print("Similarity between two sentences is:", cosim)
    return cosim
    
    
# Calling the function Doc2Vec on query and doc
Docs2Vec(query, doc)

tokenized_docs: [['apple', 'fruit'], ['apple', 'phone']]
Similarity between two sentences is: 0.48066333
Wall time: 229 ms


0.48066333

In [ ]:
# Bert embedings
# sentences1 = ["apple is fruit", "apple is phone"]
# # load the pre-trained BERT model

# from sentence_transformers import SentenceTransformer
# sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

# # encode the provided sentences to vectors
# sentence_embeddings = model.encode(sentences)


# query1 = "apple is not orange"
# query_vec = model.encode([query1])[0]

# for sent in sentences:
#   sim = cosine(query_vec, model.encode([sent])[0])
#   print("Sentence = ", sent, "; similarity = ", sim)

In [54]:
# Read data
docs = ["apple is fruit", "apple is phone"]

In [55]:
# tokenization of docs
tokenized_docs = []
for doc in docs:
    tokenized_docs.append(tokenize(doc))
tokenized_docs

[['apple', 'fruit'], ['apple', 'phone']]

In [56]:
# import gensim doc2vec
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

# tagging docs
tagged_data = [TaggedDocument(d, [i]) for i, d in enumerate(tokenized_docs)]
tagged_data

[TaggedDocument(words=['apple', 'fruit'], tags=[0]),
 TaggedDocument(words=['apple', 'phone'], tags=[1])]

In [57]:
## Train doc2vec model
model = Doc2Vec(tagged_data, vector_size = 20, window = 2, min_count = 1, epochs = 100)

'''
vector_size = Dimensionality of the feature vectors.
window = The maximum distance between the current and predicted word within a sentence.
min_count = Ignores all words with total frequency lower than this.
alpha = The initial learning rate.
'''

## Print model vocabulary
model.wv.vocab

{'apple': <gensim.models.keyedvectors.Vocab at 0x216d4fd0be0>,
 'fruit': <gensim.models.keyedvectors.Vocab at 0x216d4fd0d30>,
 'phone': <gensim.models.keyedvectors.Vocab at 0x216d4fd0e20>}

In [61]:
model.wv.n_similarity(query_tokens, doc_tokens)

0.48066333

tokenized_docs: [['apple', 'fruit'], ['apple', 'phone']]
Similarity between two sentences is: 0.48066333
Wall time: 269 ms


0.48066333

In [28]:
# s1 = 'the quick fox brown fox jumps over the lazy dog'
# s1_tag = '001'
# s2 = 'i want to burn a zero-day'
# s2_tag = '002'

# docs = []
# docs.append(TaggedDocument(words=s1.split(), tags=[s1_tag])
# docs.append(TaggedDocument(words=s2.split(), tags=[s2_tag])

# model = gensim.models.Doc2Vec(vector_size=300, window=5, min_count=5, workers=4, epochs=20)
# model.build_vocab(docs)

# print 'Start training process...'
# model.train(docs, total_examples=model.corpus_count, epochs=model.iter)

In [14]:
# def load_weModel(model, query, doc):
#     if model == 'glove':
#         glove_model = gensim.models.KeyedVectors.load("./glovemodel.mod")
#         print("Loaded glove model")
#     elif model == 'w2vec':
#         v2w_model = gensim.models.KeyedVectors.load("./w2vecmodel.mod")
#         print("Loaded w2v model")
#     elif model == 'fasttext':
#         fasttext_model300 = gensim.models.KeyedVectors.load("./fasttext_model300.mod")
#         print("Loaded fasttext_model300")
#     else:
#         print("Model not found")
        
# load_weModel('asdf')